# LLM Instrumentation Framework - Complete Demo with Reasoning Model

This notebook demonstrates the complete capabilities of the **LLM Instrumentation Framework** using a small reasoning model. We'll capture activations separately during:
1. **Prompt processing** (input phase)
2. **Reasoning generation** (output phase)

## What You'll Learn
- How to configure and initialize the instrumentation framework
- How to instrument a transformer model
- How to capture activations at different granularities
- How to analyze captured activation data
- How to work with different compression algorithms
- How to separate prompt vs. generation activations

## 1. Setup and Installation

First, let's install the required dependencies and import the necessary modules.

In [1]:
# Install required packages (uncomment if needed)
# !pip install transformers torch accelerate

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
import llm_instrumentation

# Import the instrumentation framework
from llm_instrumentation import (
    InstrumentationFramework,
    InstrumentationConfig,
    HookGranularity,
)

print("✓ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✓ All imports successful!
PyTorch version: 2.9.0+cu128
CUDA available: True


## 2. Load a Small Reasoning Model

We'll use **Qwen2.5-0.5B-Instruct**, a small but capable reasoning model that's perfect for testing. It's only 500M parameters, so it runs quickly even on CPU.

In [2]:
# Model configuration
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"

print(f"Loading model: {MODEL_NAME}")
print("This may take a minute...\n")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("✓ Tokenizer loaded")

# Load model (using CPU for compatibility, but GPU is supported)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,  # Use float32 for CPU compatibility
    device_map="auto",
    trust_remote_code=True,
)
print("✓ Model loaded")

# Print model architecture overview
print(f"\nModel architecture:")
print(f"  - Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  - Layers: {len(list(model.named_modules()))}")
print(f"  - Device: {next(model.parameters()).device}")

Loading model: Qwen/Qwen2.5-0.5B-Instruct
This may take a minute...



`torch_dtype` is deprecated! Use `dtype` instead!


✓ Tokenizer loaded
✓ Model loaded

Model architecture:
  - Total parameters: 494,032,768
  - Layers: 319
  - Device: cuda:0


## 3. Configure the Instrumentation Framework

The framework supports multiple configurations:
- **Granularity**: What to capture (full tensors, attention only, MLP only, sampled slices)
- **Compression**: Algorithm to use (lz4, zstd, none)
- **Throughput**: Target streaming rate
- **Memory**: Maximum host RAM usage

We'll create multiple configurations to demonstrate different use cases.

In [3]:
# Configuration 1: Full tensor capture with LZ4 compression
config_full = InstrumentationConfig(
    granularity=HookGranularity.FULL_TENSOR,
    compression_algorithm="lz4",  # Fast compression
    target_throughput_gbps=2.0,
    max_memory_gb=8,
)

# Configuration 2: Attention-only capture (for interpretability)
config_attention = InstrumentationConfig(
    granularity=HookGranularity.ATTENTION_ONLY,
    compression_algorithm="zstd",  # Better compression ratio
    target_throughput_gbps=2.0,
    max_memory_gb=8,
)

# Configuration 3: MLP-only capture
config_mlp = InstrumentationConfig(
    granularity=HookGranularity.MLP_ONLY,
    compression_algorithm="none",  # No compression for speed
    target_throughput_gbps=2.0,
    max_memory_gb=8,
)

print("✓ Configurations created:")
print(f"  1. Full tensor capture with LZ4")
print(f"  2. Attention-only with Zstd")
print(f"  3. MLP-only with no compression")

✓ Configurations created:
  1. Full tensor capture with LZ4
  2. Attention-only with Zstd
  3. MLP-only with no compression


## 4. Prepare Reasoning Prompts

We'll use prompts that require step-by-step reasoning to demonstrate how the model processes information differently during thinking vs. output.

In [4]:
# Define reasoning prompts
prompts = [
    """Solve this step by step:
If a train travels 120 km in 2 hours, what is its average speed?
Show your reasoning.""",
    
    """Think carefully:
What is 15% of 200? Explain each step.""",
    
    """Reason through this:
If today is Wednesday, what day will it be in 10 days?""",
]

print("Reasoning prompts prepared:")
for i, prompt in enumerate(prompts, 1):
    print(f"\n{i}. {prompt[:50]}...")

Reasoning prompts prepared:

1. Solve this step by step:
If a train travels 120 km...

2. Think carefully:
What is 15% of 200? Explain each ...

3. Reason through this:
If today is Wednesday, what d...


## 5. Demo 1: Capture Prompt Processing Activations

In this demo, we'll capture activations **only during the prompt processing phase**. We'll use the attention-only configuration to focus on how the model attends to different parts of the input.

In [5]:
# Initialize framework for prompt capture
framework_prompt = InstrumentationFramework(config_attention)

# Register the model for instrumentation
framework_prompt.instrument_model(model)
print("✓ Model instrumented for prompt capture")

# Select first prompt
test_prompt = prompts[0]
print(f"\nPrompt to process:\n{test_prompt}")

# Tokenize the prompt
inputs = tokenizer(test_prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(model.device)

print(f"\nTokenized prompt:")
print(f"  - Input shape: {input_ids.shape}")
print(f"  - Number of tokens: {input_ids.shape[1]}")
print(f"  - Device: {input_ids.device}")

# Capture activations during prompt processing
output_path_prompt = "prompt_activations.stream"

print(f"\n🔍 Capturing activations during PROMPT PROCESSING...")
print(f"   Output file: {output_path_prompt}")
print(f"   Granularity: ATTENTION_ONLY")
print(f"   Compression: zstd\n")

with framework_prompt.capture_activations(output_path_prompt):
    # Only forward pass (no generation)
    with torch.no_grad():
        outputs_prompt = model(input_ids)

print("✓ Prompt activations captured successfully!")
print(f"  - Output shape: {outputs_prompt.logits.shape}")
print(f"  - File created: {os.path.exists(output_path_prompt)}")
print(f"  - File size: {os.path.getsize(output_path_prompt):,} bytes")

✓ Model instrumented for prompt capture

Prompt to process:
Solve this step by step:
If a train travels 120 km in 2 hours, what is its average speed?
Show your reasoning.

Tokenized prompt:
  - Input shape: torch.Size([1, 31])
  - Number of tokens: 31
  - Device: cuda:0

🔍 Capturing activations during PROMPT PROCESSING...
   Output file: prompt_activations.stream
   Granularity: ATTENTION_ONLY
   Compression: zstd

✓ Prompt activations captured successfully!
  - Output shape: torch.Size([1, 31, 151936])
  - File created: True
  - File size: 0 bytes


## 6. Demo 2: Capture Generation (Reasoning) Activations

Now we'll capture activations **during the generation phase**, when the model is actively reasoning and producing output tokens. This uses full tensor capture to get complete information.

In [ ]:
# Initialize framework for generation capture
framework_generation = InstrumentationFramework(config_full)

# Register the model
framework_generation.instrument_model(model)
print("✓ Model instrumented for generation capture")

# Prepare the same prompt
inputs = tokenizer(test_prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(model.device)

# Capture activations during generation
output_path_generation = "generation_activations.stream"

print(f"\n🔍 Capturing activations during GENERATION (Reasoning)...")
print(f"   Output file: {output_path_generation}")
print(f"   Granularity: FULL_TENSOR")
print(f"   Compression: lz4")
print(f"   Max new tokens: 1024\n")

with framework_generation.capture_activations(output_path_generation):
    # Generate tokens (reasoning phase)
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )

# Decode the generated text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("✓ Generation activations captured successfully!")
print(f"  - File created: {os.path.exists(output_path_generation)}")
print(f"  - File size: {os.path.getsize(output_path_generation):,} bytes")
print(f"\n📝 Generated response:\n")
print(f"{generated_text}")

✓ Model instrumented for generation capture

🔍 Capturing activations during GENERATION (Reasoning)...
   Output file: generation_activations.stream
   Granularity: FULL_TENSOR
   Compression: lz4
   Max new tokens: 512

✓ Generation activations captured successfully!
  - File created: True
  - File size: 64,953,573 bytes

📝 Generated response:

Solve this step by step:
If a train travels 120 km in 2 hours, what is its average speed?
Show your reasoning. Let's solve this problem step by step:

Step 1: Identify the given information
- Distance traveled (d) = 120 km
- Time taken (t) = 2 hours

Step 2: Recall the formula for average speed
Average speed (v_avg) = d/t

Step 3: Plug in the values into the formula
v_avg = d/t = 120 km / 2 h

Step 4: Simplify the equation
v_avg = 60 km/h

Therefore, the average speed of the train is 60 kilometers per hour.

The reasoning process shows that we used the definition of average speed to find the result directly from the given distance and time. This

## 7. Demo 3: Per-Token Capture (Reasoning Analysis)

Now let's demonstrate the **per-token tracking feature**. This is particularly useful for analyzing how activations change with each generated token during reasoning. We'll manually generate tokens one-by-one and track each one.

The `track_per_token=True` flag enables per-token metadata collection, which is saved to a separate JSON file.

In [ ]:
# Initialize framework for per-token capture
framework_per_token = InstrumentationFramework(config_full)
framework_per_token.instrument_model(model)

print("✓ Model instrumented for per-token capture")

# Select a reasoning prompt
test_prompt_token = prompts[0]
print(f"\nPrompt: {test_prompt_token[:80]}...\n")

# Tokenize the prompt
inputs = tokenizer(test_prompt_token, return_tensors="pt")
input_ids = inputs["input_ids"].to(model.device)

output_path_per_token = "generation_per_token.stream"

print(f"🔍 Capturing activations with per-token tracking...")
print(f"   Output file: {output_path_per_token}")
print(f"   Token metadata: {output_path_per_token.replace('.stream', '_tokens.json')}")
print(f"   Max tokens: 100\n")

# Capture with per-token tracking enabled
with framework_per_token.capture_activations(output_path_per_token, track_per_token=True) as tracker:
    current_ids = input_ids
    generated_tokens = []
    
    for step in range(100):
        # Forward pass to get next token
        with torch.no_grad():
            outputs = model(current_ids)
            next_token_logits = outputs.logits[:, -1, :]
            next_token = next_token_logits.argmax(dim=-1, keepdim=True)
        
        # Record the token with the tracker
        token_id = next_token[0].item()
        token_text = tokenizer.decode(next_token[0])
        tracker.record_token(
            token_id=token_id,
            token_text=token_text
        )
        
        generated_tokens.append(token_text)
        
        # Append to input sequence
        current_ids = torch.cat([current_ids, next_token], dim=-1)
        
        # Stop if we hit EOS token
        if token_id == tokenizer.eos_token_id:
            print(f"✓ Generation completed at token {step + 1} (EOS reached)")
            break
    else:
        print(f"✓ Generation completed after {step + 1} tokens (max reached)")

# Decode the full generated sequence
full_text = tokenizer.decode(current_ids[0], skip_special_tokens=True)

print(f"\n📝 Generated {len(generated_tokens)} tokens")
print(f"  - File created: {os.path.exists(output_path_per_token)}")
print(f"  - File size: {os.path.getsize(output_path_per_token):,} bytes")
print(f"\n📄 Full generated text:\n{full_text[:500]}...")

# Analyze with token metadata using the helper function
from llm_instrumentation import analyze_activations_with_tokens

analysis_per_token = analyze_activations_with_tokens(output_path_per_token, framework_per_token)

print(f"\n📊 Per-Token Analysis:")
print(f"  - Total tokens generated: {analysis_per_token.get('token_metadata', {}).get('total_tokens', 0)}")
print(f"  - Total packets captured: {analysis_per_token['packets']:,}")

if 'packets_per_token' in analysis_per_token:
    print(f"  - Average packets per token: {analysis_per_token['packets_per_token']:.2f}")
    print(f"  - Average bytes per token: {analysis_per_token['bytes_per_token']:,.0f}")

if 'token_metadata' in analysis_per_token:
    print(f"\n📋 First 10 tokens generated:")
    for token_info in analysis_per_token['token_metadata']['tokens'][:10]:
        print(f"    [{token_info['token_index']}] Token ID {token_info['token_id']}: '{token_info['token_text']}'")
    
    print(f"\n💡 Use Case:")
    print(f"  This per-token capture allows you to:")
    print(f"  - Correlate activations with specific reasoning steps")
    print(f"  - Analyze how model confidence evolves during generation")
    print(f"  - Identify patterns in activation changes between tokens")
    print(f"  - Debug specific points in the reasoning chain")

## 8. Analyze Captured Activations

Now let's analyze both activation streams to see the differences between prompt processing and generation phases.

In [7]:
# Analyze prompt activations
print("\n" + "="*60)
print("ANALYSIS: PROMPT PROCESSING ACTIVATIONS")
print("="*60)

analysis_prompt = framework_prompt.analyze_activations(output_path_prompt)

print(f"\n📊 Overall Statistics:")
print(f"  - Stream path: {analysis_prompt['stream_path']}")
print(f"  - Compression: {analysis_prompt['compression']}")
print(f"  - Total packets: {analysis_prompt['packets']:,}")
print(f"  - Total compressed bytes: {analysis_prompt['total_compressed_bytes']:,}")

print(f"\n📋 Per-Layer Breakdown:")
for layer_name, stats in sorted(analysis_prompt['per_layer'].items()):
    if layer_name:  # Skip empty names
        avg_size = stats['bytes'] / stats['count'] if stats['count'] > 0 else 0
        print(f"  {layer_name:30s} → {stats['count']:3d} packets, {stats['bytes']:10,} bytes (avg: {avg_size:,.0f} bytes/packet)")

# Analyze generation activations
print("\n" + "="*60)
print("ANALYSIS: GENERATION (REASONING) ACTIVATIONS")
print("="*60)

analysis_generation = framework_generation.analyze_activations(output_path_generation)

print(f"\n📊 Overall Statistics:")
print(f"  - Stream path: {analysis_generation['stream_path']}")
print(f"  - Compression: {analysis_generation['compression']}")
print(f"  - Total packets: {analysis_generation['packets']:,}")
print(f"  - Total compressed bytes: {analysis_generation['total_compressed_bytes']:,}")

print(f"\n📋 Per-Layer Breakdown (top 10 by packet count):")
sorted_layers = sorted(analysis_generation['per_layer'].items(), 
                      key=lambda x: x[1]['count'], 
                      reverse=True)
for layer_name, stats in sorted_layers[:10]:
    if layer_name:
        avg_size = stats['bytes'] / stats['count'] if stats['count'] > 0 else 0
        print(f"  {layer_name:30s} → {stats['count']:3d} packets, {stats['bytes']:10,} bytes (avg: {avg_size:,.0f} bytes/packet)")


ANALYSIS: PROMPT PROCESSING ACTIVATIONS

📊 Overall Statistics:
  - Stream path: prompt_activations.stream
  - Compression: zstd
  - Total packets: 0
  - Total compressed bytes: 0

📋 Per-Layer Breakdown:

ANALYSIS: GENERATION (REASONING) ACTIVATIONS

📊 Overall Statistics:
  - Stream path: generation_activations.stream
  - Compression: lz4
  - Total packets: 153
  - Total compressed bytes: 46,436,630

📋 Per-Layer Breakdown (top 10 by packet count):
  lm_head                        → 153 packets, 46,436,630 bytes (avg: 303,507 bytes/packet)


## 9. Compare Prompt vs. Generation Phases

Let's create a comparison to understand how the model behaves differently during prompt processing vs. generation.

In [8]:
print("\n" + "="*60)
print("COMPARISON: PROMPT vs GENERATION")
print("="*60)

comparison = {
    "Metric": [
        "Total Packets",
        "Total Compressed Bytes",
        "Unique Layers",
        "Compression Algorithm",
        "Granularity",
    ],
    "Prompt Processing": [
        f"{analysis_prompt['packets']:,}",
        f"{analysis_prompt['total_compressed_bytes']:,}",
        f"{len(analysis_prompt['per_layer'])}",
        analysis_prompt['compression'],
        "ATTENTION_ONLY",
    ],
    "Generation (Reasoning)": [
        f"{analysis_generation['packets']:,}",
        f"{analysis_generation['total_compressed_bytes']:,}",
        f"{len(analysis_generation['per_layer'])}",
        analysis_generation['compression'],
        "FULL_TENSOR",
    ],
}

# Print comparison table
print(f"\n{'Metric':<30} {'Prompt Processing':<30} {'Generation'}")
print("-" * 80)
for i, metric in enumerate(comparison["Metric"]):
    print(f"{metric:<30} {comparison['Prompt Processing'][i]:<30} {comparison['Generation (Reasoning)'][i]}")

# Calculate ratio
if analysis_prompt['packets'] > 0:
    packet_ratio = analysis_generation['packets'] / analysis_prompt['packets']
    byte_ratio = analysis_generation['total_compressed_bytes'] / analysis_prompt['total_compressed_bytes']
    
    print(f"\n📈 Generation/Prompt Ratios:")
    print(f"  - Packet ratio: {packet_ratio:.2f}x more packets during generation")
    print(f"  - Data ratio: {byte_ratio:.2f}x more data during generation")
    print(f"\n💡 Interpretation:")
    print(f"  Generation captures {packet_ratio:.1f}x more activations because:")
    print(f"  1. Multiple forward passes (one per generated token)")
    print(f"  2. Full tensor capture vs attention-only")
    print(f"  3. The model processes {generated_ids.shape[1] - input_ids.shape[1]} new tokens")


COMPARISON: PROMPT vs GENERATION

Metric                         Prompt Processing              Generation
--------------------------------------------------------------------------------
Total Packets                  0                              153
Total Compressed Bytes         0                              46,436,630
Unique Layers                  0                              1
Compression Algorithm          zstd                           lz4
Granularity                    ATTENTION_ONLY                 FULL_TENSOR


## 10. Demo 4: MLP-Only Capture for Feature Analysis

Let's demonstrate MLP-only capture, which is useful for understanding how the model transforms representations in the feedforward layers.

In [9]:
# Initialize framework for MLP capture
framework_mlp = InstrumentationFramework(config_mlp)
framework_mlp.instrument_model(model)

print("✓ Model instrumented for MLP-only capture")

# Use a different prompt
mlp_prompt = prompts[1]
print(f"\nPrompt: {mlp_prompt}\n")

inputs = tokenizer(mlp_prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(model.device)

output_path_mlp = "mlp_activations.stream"

print(f"🔍 Capturing MLP activations...")
print(f"   Granularity: MLP_ONLY")
print(f"   Compression: none\n")

with framework_mlp.capture_activations(output_path_mlp):
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids,
            max_new_tokens=50,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
        )

generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("✓ MLP activations captured!")
print(f"  - File size: {os.path.getsize(output_path_mlp):,} bytes")
print(f"\n📝 Generated response:\n{generated_text}")

# Analyze MLP activations
analysis_mlp = framework_mlp.analyze_activations(output_path_mlp)

print(f"\n📊 MLP Analysis:")
print(f"  - Total packets: {analysis_mlp['packets']:,}")
print(f"  - Total bytes (uncompressed): {analysis_mlp['total_compressed_bytes']:,}")
print(f"  - Unique MLP layers captured: {len(analysis_mlp['per_layer'])}")

if analysis_mlp['per_layer']:
    print(f"\n📋 MLP Layers:")
    for layer_name in sorted(analysis_mlp['per_layer'].keys())[:5]:
        if layer_name:
            print(f"  - {layer_name}")

✓ Model instrumented for MLP-only capture

Prompt: Think carefully:
What is 15% of 200? Explain each step.

🔍 Capturing MLP activations...
   Granularity: MLP_ONLY
   Compression: none

✓ MLP activations captured!
  - File size: 0 bytes

📝 Generated response:
Think carefully:
What is 15% of 200? Explain each step. To find what 15% of 200 is, follow these steps:

1. **Understand the problem**: We need to calculate 15% of a number. In mathematical terms, this is represented as \( 15

📊 MLP Analysis:
  - Total packets: 0
  - Total bytes (uncompressed): 0
  - Unique MLP layers captured: 0


## 11. Verification: Compare Compression Algorithms

Let's verify compression effectiveness by comparing file sizes across different algorithms.

In [10]:
# Test all compression algorithms with the same prompt and settings
compression_results = {}
test_prompt_short = "What is 2 + 2? Explain."
inputs_test = tokenizer(test_prompt_short, return_tensors="pt")
input_ids_test = inputs_test["input_ids"].to(model.device)

print("\n🧪 Testing compression algorithms...\n")

for algo in ["none", "lz4", "zstd"]:
    config_test = InstrumentationConfig(
        granularity=HookGranularity.FULL_TENSOR,
        compression_algorithm=algo,
        target_throughput_gbps=2.0,
        max_memory_gb=8,
    )
    
    framework_test = InstrumentationFramework(config_test)
    framework_test.instrument_model(model)
    
    output_path_test = f"compression_test_{algo}.stream"
    
    print(f"Testing {algo:6s} compression...", end=" ")
    
    with framework_test.capture_activations(output_path_test):
        with torch.no_grad():
            _ = model.generate(
                input_ids_test,
                max_new_tokens=30,
                pad_token_id=tokenizer.eos_token_id,
            )
    
    file_size = os.path.getsize(output_path_test)
    compression_results[algo] = file_size
    print(f"✓ {file_size:,} bytes")

# Calculate compression ratios
print("\n" + "="*60)
print("COMPRESSION COMPARISON")
print("="*60)

baseline = compression_results["none"]
print(f"\n{'Algorithm':<10} {'Size (bytes)':<15} {'Compression Ratio':<20} {'Reduction'}")
print("-" * 70)

for algo, size in compression_results.items():
    ratio = baseline / size if size > 0 else 0
    reduction = (1 - size/baseline) * 100 if baseline > 0 else 0
    print(f"{algo:<10} {size:<15,} {ratio:<20.2f}x {reduction:>5.1f}% reduction")

print(f"\n💡 Best compression: {min(compression_results, key=compression_results.get)}")
print(f"💡 Fastest (no compression): none")


🧪 Testing compression algorithms...

Testing none   compression... ✓ 9,116,550 bytes
Testing lz4    compression... ✓ 9,106,360 bytes
Testing zstd   compression... ✓ 8,121,115 bytes

COMPRESSION COMPARISON

Algorithm  Size (bytes)    Compression Ratio    Reduction
----------------------------------------------------------------------
none       9,116,550       1.00                x   0.0% reduction
lz4        9,106,360       1.00                x   0.1% reduction
zstd       8,121,115       1.12                x  10.9% reduction

💡 Best compression: zstd
💡 Fastest (no compression): none


## 12. Summary and Verification

Let's verify that everything worked correctly and summarize what we've learned.

In [11]:
print("\n" + "="*60)
print("VERIFICATION CHECKLIST")
print("="*60)

checks = [
    ("Model loaded successfully", model is not None),
    ("Prompt activations captured", os.path.exists(output_path_prompt)),
    ("Generation activations captured", os.path.exists(output_path_generation)),
    ("Per-token activations captured", os.path.exists(output_path_per_token)),
    ("MLP activations captured", os.path.exists(output_path_mlp)),
    ("Prompt analysis completed", analysis_prompt['packets'] > 0),
    ("Generation analysis completed", analysis_generation['packets'] > 0),
    ("Per-token analysis completed", 'token_metadata' in analysis_per_token),
    ("MLP analysis completed", analysis_mlp['packets'] > 0),
    ("Compression test completed", len(compression_results) == 3),
    ("Model generated valid output", len(generated_text) > len(test_prompt)),
]

all_passed = True
for check_name, result in checks:
    status = "✓" if result else "✗"
    print(f"{status} {check_name}")
    if not result:
        all_passed = False

print("\n" + "="*60)
if all_passed:
    print("✓ ALL CHECKS PASSED - Framework is working correctly!")
else:
    print("✗ Some checks failed - please review the output above")
print("="*60)

# Print summary
print("\n" + "="*60)
print("SUMMARY: What You've Learned")
print("="*60)
print("""
✓ How to configure the instrumentation framework with different settings
✓ How to instrument a transformer model for activation capture
✓ How to capture activations separately for:
  - Prompt processing (input phase)
  - Generation/reasoning (output phase)
  - Per-token tracking for fine-grained analysis
✓ How to use different granularities:
  - FULL_TENSOR: Complete activation capture
  - ATTENTION_ONLY: Focus on attention mechanisms
  - MLP_ONLY: Focus on feedforward layers
✓ How to compare compression algorithms:
  - none: Fastest, largest files
  - lz4: Fast compression, good ratio
  - zstd: Best compression, slower
✓ How to analyze captured activation streams
✓ How to interpret per-layer statistics
✓ How to track tokens during generation for reasoning analysis
""")

print("="*60)
print("FILES GENERATED")
print("="*60)
print(f"""
1. {output_path_prompt} ({os.path.getsize(output_path_prompt):,} bytes)
   → Attention-only activations from prompt processing
   
2. {output_path_generation} ({os.path.getsize(output_path_generation):,} bytes)
   → Full tensor activations from reasoning/generation
   
3. {output_path_mlp} ({os.path.getsize(output_path_mlp):,} bytes)
   → MLP-only activations (uncompressed)
   
4. compression_test_*.stream (3 files)
   → Compression algorithm comparison
""")

print("\n" + "="*60)
print("NEXT STEPS")
print("="*60)
print("""
Now you can:
1. Use these activation streams for downstream analysis
2. Feed them into interpretability tools (SAE, causal graphs)
3. Correlate activations with model behavior
4. Optimize model performance based on activation patterns
5. Compare activations across different prompts/models

For more information, see the documentation:
- docs/API.md - Complete API reference
- docs/STREAM_FORMAT.md - Stream format specification
- docs/ARCHITECTURE.md - Framework architecture
""")


VERIFICATION CHECKLIST
✓ Model loaded successfully
✓ Prompt activations captured
✓ Generation activations captured
✓ MLP activations captured
✗ Prompt analysis completed
✓ Generation analysis completed
✗ MLP analysis completed
✓ Compression test completed
✓ Model generated valid output

✗ Some checks failed - please review the output above

SUMMARY: What You've Learned

✓ How to configure the instrumentation framework with different settings
✓ How to instrument a transformer model for activation capture
✓ How to capture activations separately for:
  - Prompt processing (input phase)
  - Generation/reasoning (output phase)
✓ How to use different granularities:
  - FULL_TENSOR: Complete activation capture
  - ATTENTION_ONLY: Focus on attention mechanisms
  - MLP_ONLY: Focus on feedforward layers
✓ How to compare compression algorithms:
  - none: Fastest, largest files
  - lz4: Fast compression, good ratio
  - zstd: Best compression, slower
✓ How to analyze captured activation streams
✓ 

## 13. Cleanup (Optional)

Uncomment and run this cell to remove generated files.

In [12]:
# import os

# files_to_remove = [
#     output_path_prompt,
#     output_path_generation,
        "generation_per_token.stream",
        "generation_per_token_tokens.json",
#     output_path_mlp,
#     "compression_test_none.stream",
#     "compression_test_lz4.stream",
#     "compression_test_zstd.stream",
# ]

# for file_path in files_to_remove:
#     if os.path.exists(file_path):
#         os.remove(file_path)
#         print(f"Removed: {file_path}")

# print("\n✓ Cleanup complete!")